In [ ]:
import json
import csv
import os

In [ ]:
# JSON 파일들이 담긴 디렉토리 경로
input_directory = 'out/'
output_directory = 'out_csv/'

In [ ]:
# 출력 디렉토리 생성 (존재하지 않는 경우)
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [ ]:
# CSV 파일 헤더 정의 (필요한 컬럼만 유지)
csv_headers = ["ID", "성별", "나이", "학년", "유형구분", "가정환경", "상담일자", "평가일시", "작성자(상담사)", 
               "상호작용 특성(종합)", "긴장 수준(종합)", "행동 특성(종합)", "위기단계", "합계점수", "학대의심", 
               "행동특성 점수", "임상가 종합소견"]

In [ ]:
# 디렉토리 내의 모든 JSON 파일 변환
for filename in os.listdir(input_directory):
    if filename.endswith('.json'):
        json_file_path = os.path.join(input_directory, filename)
        csv_file_path = os.path.join(output_directory, filename.replace('.json', '.csv'))

        # JSON 파일 읽기
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)

        # CSV 파일 작성
        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)

            # CSV 헤더 작성 (ID와 위기 단계만 유지, 이후 열 이름 없음)
            csv_headers = ['ID', '위기 단계', '합계 점수'] #------- 이 부분에 필요한 컬럼 추가
            for i in range(1, 101):
                csv_headers.append(f'Q{i}')
                csv_headers.append(f'A{i}')
    
            csv_writer.writerow(csv_headers) 


            # info 데이터에서 ID와 위기 단계 값만 가져오기
            row_data = [
                data['info'].get('ID', ''),             # 'ID' 값
                data['info'].get('위기단계', ''),        # '위기 단계' 값
                data['info'].get('합계점수', '')        # '합계 점수' 값
                                                        # ------- 이 부분에서 값 가져오기
            ]

            # list 부분 추가
            list_data = []
            for item in data.get('list', []):
                for sub_item in item.get('list', []):
                    for audio in sub_item.get('audio', []):
                        list_data.append(f"{audio['type']}: {audio['text']}")

            # 추가 데이터를 이어붙임 (열 이름 없이 데이터만 추가)
            row_data.extend(list_data)
            
            # 한 행으로 작성 (열 이름 없이 데이터 작성)
            csv_writer.writerow(row_data)

        print(f"{filename} 변환 완료, {csv_file_path}에 저장되었습니다.")


In [ ]:
import pandas as pd
from glob import glob

# 폴더 내의 모든 CSV 파일 목록을 가져옴
file_names = glob("out_csv/*.csv")  # 파일 경로에 맞게 수정 필요

# 빈 데이터프레임 생성
total = pd.DataFrame()

# 각 파일을 읽어서 total 데이터프레임에 합침
for file_name in file_names:
    temp = pd.read_csv(file_name, encoding='utf-8')  # 필요한 구분자 또는 인코딩을 사용
    total = pd.concat([total, temp], ignore_index=True)

# 최종적으로 합쳐진 파일을 저장
total.to_csv("out_csv/total_combined_output.csv", index=False)
